In [1]:
using JuMP, Gurobi, CSV

In [2]:
pixels = CSV.read("output_ghana5_small.csv", header = 1)
pixelsSize = size(pixels)   #tuple representing dimensions of country
xMax = pixelsSize[1]
yMax = pixelsSize[2]
print(xMax, ", ", yMax)

52, 40

In [3]:
totalPop = sum(sum(pixels[i,j] for i in 1:xMax) for j in 1:yMax)
percentage = 0.95
print(totalPop)

2.7408841856313188e7

In [4]:
model = Model(with_optimizer(Gurobi.Optimizer))

Academic license - for non-commercial use only


A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Gurobi

In [5]:
@variable(model, S[1:xMax, 1:yMax], Bin)   # S[i,j] = 1 if there is a small access pt at (i,j)
@variable(model, L[1:xMax, 1:yMax], Bin)   # L[i,j] = 1 if there is a large access pt at (i,j)
@variable(model, R[1:xMax, 1:yMax], Bin);

In [6]:
TS = sum(sum(S[i,j] for j in 1:yMax) for i in 1:xMax)   # total number of small access pts
TL = sum(sum(L[i,j] for j in 1:yMax) for i in 1:xMax);  # total number of large access pts

In [7]:
@objective(model, Min, 150*TS+250*TL);   # minimize total cost of access pts

In [8]:
ds = 3       # 5
dl = 8       # 15
for i in 1:xMax
    for j in 1:yMax
        @constraint(model, R[i,j] <= sum(sum(S[i+dx, j+dy]
                                     for dy in max(trunc(Int, -sqrt(ds^2-dx^2)), 1-j) : min(yMax-j, trunc(Int, sqrt(ds^2-dx^2))))
                                     for dx in max(-ds, 1-i) : min(ds, xMax-i)) +
                                     sum(sum(L[i+dx, j+dy]
                                     for dy in max(trunc(Int, -sqrt(dl^2-dx^2)), 1-j) : min(yMax-j, trunc(Int, sqrt(dl^2-dx^2))))
                                     for dx in max(-dl, 1-i) : min(dl, xMax-i)))     
        end #R[i,j] must be <= the sum of the access points within range of that pixel. ie it will be 0 if there are no access points in range; 1 if there is at least 1 access pt in range
end                           

In [9]:
@constraint(model, sum(sum(pixels[i,j]*R[i,j] for i in 1:xMax) for j in 1:yMax) >= percentage*totalPop);

In [10]:
# Solving the optimization problem  
optimize!(model)

Academic license - for non-commercial use only
Optimize a model with 2081 rows, 6240 columns and 411516 nonzeros
Variable types: 0 continuous, 6240 integer (6240 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+06]
  Objective range  [2e+02, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+07, 3e+07]
Found heuristic solution: objective 30750.000000
Found heuristic solution: objective 8500.0000000
Presolve removed 684 rows and 2138 columns
Presolve time: 0.91s
Presolved: 1397 rows, 4102 columns, 254658 nonzeros
Found heuristic solution: objective 16500.000000
Found heuristic solution: objective 5650.0000000
Variable types: 0 continuous, 4102 integer (4102 binary)

Root relaxation: objective 1.744392e+03, 2061 iterations, 0.34 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1744.39234    0  345 5650.00000 1744.39234  69.1%     -    1s
H 

In [11]:
# Printing the optimal solutions obtained
S_opt = value.(S)
L_opt = value.(L)
println("S = ", S_opt)
println("L = ", L_opt)
println("objective = ", objective_value(model))

S = [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -0.0 -0.0 -0.0 -0.0 -0.0 -0.0 -0.0 -0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -0.0 -0.0 -0.0 -0.0 -0.0 -0.0 -0.0 -0.0 -0.0 -0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.

L = [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -0.0 -0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -0.0 -0.0 -0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.

objective = 1750.0


In [12]:
for i in 1:xMax
    for j in 1:yMax
        if S_opt[i, j] > 0.0
            println("S:", S_opt[i, j])
            println(i, ", ", j)
        end
        if L_opt[i, j] > 0.0
            println("L:", L_opt[i, j])    
            println(i, ", ", j)
        end
    end
end

L:1.0
13 8
L:1.0
13 36
L:1.0
14 26
L:1.0
28 32
L:1.0
31 5
L:1.0
38 17
L:1.0
41 32
